<a href="https://colab.research.google.com/github/mayarachew/NLP/blob/main/Competi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import math

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
df_train_raw = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/training.csv')
df_test_raw = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/test.csv')

In [12]:
df_train_raw.head()

,Id,text,Expected
0,0,"Ótima localização, bom atendimento, bom sinal ...",neutro
1,1,"O hotel é bem localizado e silencioso, o que c...",neutro
2,2,O hotel é muito bom. O atendimento é excelente...,positivo
3,3,"Hotel antigo, mal conservado e mal reformado, ...",negativo
4,4,"Hotel bem localizado, perto de dois shopping, ...",positivo


In [13]:
df_test_raw.head()

,Id,text
0,5000,O Naoum Hotel está lozalizado no centro comerc...
1,5001,Ótima localização para quem vai tirar o visto ...
2,5002,"Esta é a segunda vez que fico neste hotel, dev..."
3,5003,"Boa localização com café da manhã razoável, To..."
4,5004,"Hotel Naoum é muito bom, ótimo serviço, muito ..."


In [99]:
def preprocessing(text):
    # regex
    pp_text = re.sub(r':\)|\(:', 'happy', text)
    pp_text = re.sub(r':\(|\):', 'sad', pp_text)
    pp_text = re.sub(r':D', 'excited', pp_text)
    pp_text = re.sub(r'D:', 'sorrowful', pp_text)
    pp_text = re.sub(r'#', '', pp_text)
    pp_text = re.sub(r'\$[\d.,]*', 'price', pp_text)
    pp_text = re.sub(r'[^ ]*(\’m)', 'I am', pp_text)
    pp_text = re.sub(r'\’t', ' not', pp_text)
    pp_text = re.sub(r'its', ' it’s', pp_text)
    pp_text = re.sub(r'http:[^ ]*', '', pp_text)
    pp_text = re.sub(r'@[^ ]*', '', pp_text)
    pp_text = re.sub(r'!', ' important ', pp_text)
    pp_text = re.sub(r'\.\.\.', ' etc ', pp_text)
    pp_text = re.sub(r'[^\w\s]', '', pp_text)
    pp_text = re.sub(r'  *', ' ', pp_text)
    pp_text = re.sub(r'\n', ' ', pp_text)
    pp_text = pp_text.lower()

    # tokenization
    pp_token_text = word_tokenize(pp_text)

    # remove stopwords
    pp_token_text = [word for word in pp_token_text if word not in stopwords.words('portuguese')]

    # lemmatization
    wl = WordNetLemmatizer()
    pp_token_text = [wl.lemmatize(word) for word in pp_token_text]
    return pp_text

In [252]:
texts_list = []
corpus = ''
num_samples = 500

for text in df_train_raw['text'][:num_samples]:
  pp_text = preprocessing(text)
  texts_list.append(pp_text)
  corpus += pp_text+''

tokens_train = corpus.split()
vocab_train = sorted(set(tokens_train))
all_vocab_train = sorted(tokens_train)

df_train = pd.DataFrame()
df_train['Expected'] = df_train_raw['Expected'][:num_samples]
df_train['text'] = texts_list

df_train.head(3)

,Expected,text
0,neutro,ótima localização bom atendimento bom sinal de...
1,neutro,o hotel é bem localizado e silencioso o que co...
2,positivo,o hotel é muito bom o atendimento é excelente ...


In [253]:
df_train

,Expected,text
0,neutro,ótima localização bom atendimento bom sinal de...
1,neutro,o hotel é bem localizado e silencioso o que co...
2,positivo,o hotel é muito bom o atendimento é excelente ...
3,negativo,hotel antigo mal conservado e mal reformado ap...
4,positivo,hotel bem localizado perto de dois shopping qu...
...,...,...
495,NaN,ótima relação custo x benefício important hote...
496,NaN,a média de preços de hoteis decentes em brasil...
497,NaN,muito bom o atendimento important excelente ac...
498,NaN,conforto higiene e excelente custo x benefício...


In [206]:
texts_list = []
corpus = ''

for text in df_test_raw['text']:
  pp_text = preprocessing(text)
  texts_list.append(pp_text)
  corpus += pp_text+''

tokens_test = corpus.split()
vocab_test = sorted(set(tokens_test))
all_vocab_test = sorted(tokens_test)

df_test = pd.DataFrame()
df_test['text'] = texts_list

df_test.head(3)

,text
0,o naoum hotel está lozalizado no centro comerc...
1,ótima localização para quem vai tirar o visto ...
2,esta é a segunda vez que fico neste hotel devi...


In [207]:
vocab = set(vocab_train + vocab_test)

In [208]:
df = pd.concat([df_train, df_test])
df

,Expected,text
0,neutro,ótima localização bom atendimento bom sinal de...
1,neutro,o hotel é bem localizado e silencioso o que co...
2,positivo,o hotel é muito bom o atendimento é excelente ...
3,negativo,hotel antigo mal conservado e mal reformado ap...
4,positivo,hotel bem localizado perto de dois shopping qu...
...,...,...
395,NaN,a localização o faz um ótimo hotel para se est...
396,NaN,há uma preocupação com a limpeza a com o confo...
397,NaN,o mercure lider é uma boa opção à negócios poi...
398,NaN,o hotel é bem localizado muito perto das princ...


tf-idf

In [209]:
def create_tfidf(df):
  tf = {}
  N = {}

  for idx, text in enumerate(df['text']):
    tf[f'sentenca {idx+1}'] = dict()
    N[f'sentenca {idx+1}'] = len(text.split())
    for word in text.split():
      if word in tf[f'sentenca {idx+1}']:
        tf[f'sentenca {idx+1}'][word] += 1
      else:
        tf[f'sentenca {idx+1}'][word] = 1

  for text in tf:
    for word in tf[text]:
      tf[text][word] /= N[text]

  bow_contagem = dict()

  for idx, text in enumerate(df['text']):
    bow_contagem[f'sentenca {idx+1}'] = dict()
    for word in text.split():
      if word in bow_contagem[f'sentenca {idx+1}']:
          bow_contagem[f'sentenca {idx+1}'][word] += 1
      else:
          bow_contagem[f'sentenca {idx+1}'][word] = 1

  idf = {}

  for word in vocab:
    idf[word] = 0
    # count how many documents have this word
    for text in bow_contagem:
      if word in bow_contagem[text]:
        idf[word] += 1

  for word in idf:
    idf[word] = math.log(len(df['text'])/idf[word])

  tfidf = {}

  for text in tf:
      tfidf[text] = dict()
      for word in tf[text]:
          tfidf[text][word] = tf[text][word]*idf[word]

  tfidf = pd.DataFrame().from_records(tfidf).fillna(0).T

  return tfidf

In [210]:
tfidf = create_tfidf(df)
tfidf.head()

,ótima,localização,bom,atendimento,sinal,de,wifi,gratuito,preço,o,...,educadas,equilíbrio,vencida,ouvi,vencido,arrumaram,adiantado,diáriaimpostotaxa,participação,empossava
sentenca 1,0.020032,0.010681,0.043738,0.013005,0.043609,0.012939,0.025286,0.037690,0.018931,0.016390,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sentenca 10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sentenca 100,0.000000,0.000000,0.000000,0.020523,0.000000,0.004084,0.000000,0.000000,0.000000,0.021555,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sentenca 101,0.000000,0.026970,0.044175,0.000000,0.000000,0.000000,0.000000,0.095167,0.000000,0.006897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sentenca 102,0.000000,0.000000,0.000000,0.000000,0.000000,0.007260,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [239]:
df_train

,Expected,text
0,neutro,ótima localização bom atendimento bom sinal de...
1,neutro,o hotel é bem localizado e silencioso o que co...
2,positivo,o hotel é muito bom o atendimento é excelente ...
3,negativo,hotel antigo mal conservado e mal reformado ap...
4,positivo,hotel bem localizado perto de dois shopping qu...
...,...,...
495,NaN,ótima relação custo x benefício important hote...
496,NaN,a média de preços de hoteis decentes em brasil...
497,NaN,muito bom o atendimento important excelente ac...
498,NaN,conforto higiene e excelente custo x benefício...


In [248]:
category_dict = dict()

for idx, text in enumerate(df_train['text'][0:num_samples]):
  category_dict[f'sentenca {idx}'] = df_train['Expected'][idx]

In [212]:
tfidf_train = tfidf[0:num_samples]
tfidf_test = tfidf[num_samples:]

In [221]:
def cosine_similarity(vetor1, vetor2):
    inner_prod = 0

    for i,valor in enumerate(vetor1):
        inner_prod += valor*vetor2[i]

    norma_vetor1 = math.sqrt(sum([x**2 for x in vetor1])) 
    norma_vetor2 = math.sqrt(sum([x**2 for x in vetor2])) 
    
    return inner_prod/(norma_vetor1*norma_vetor2)

In [251]:
category_dict

{'sentenca 0': 'neutro',
 'sentenca 1': 'neutro',
 'sentenca 10': 'positivo',
 'sentenca 100': nan,
 'sentenca 101': nan,
 'sentenca 102': nan,
 'sentenca 103': nan,
 'sentenca 104': nan,
 'sentenca 105': nan,
 'sentenca 106': nan,
 'sentenca 107': nan,
 'sentenca 108': nan,
 'sentenca 109': nan,
 'sentenca 11': 'neutro',
 'sentenca 110': nan,
 'sentenca 111': nan,
 'sentenca 112': nan,
 'sentenca 113': nan,
 'sentenca 114': nan,
 'sentenca 115': nan,
 'sentenca 116': nan,
 'sentenca 117': nan,
 'sentenca 118': nan,
 'sentenca 119': nan,
 'sentenca 12': 'positivo',
 'sentenca 120': nan,
 'sentenca 121': nan,
 'sentenca 122': nan,
 'sentenca 123': nan,
 'sentenca 124': nan,
 'sentenca 125': nan,
 'sentenca 126': nan,
 'sentenca 127': nan,
 'sentenca 128': nan,
 'sentenca 129': nan,
 'sentenca 13': 'positivo',
 'sentenca 130': nan,
 'sentenca 131': nan,
 'sentenca 132': nan,
 'sentenca 133': nan,
 'sentenca 134': nan,
 'sentenca 135': nan,
 'sentenca 136': nan,
 'sentenca 137': nan,
 'se

In [250]:
all_top_10_lists = []

for idx, row in tfidf_test.iterrows():
  top_list = []
  for idx_to_compare, row_to_compare in tfidf_train.iterrows():
    print(idx_to_compare)
    top_list.append({'idx':idx,
                     'idx to compare':idx_to_compare, 
                     'category': category_dict[idx_to_compare],
                     'cosine similarity':cosine_similarity(tfidf_test.loc[idx].to_numpy(), tfidf_train.loc[idx_to_compare].to_numpy())})
  top_list = sorted(top_list, key=lambda d: d['cosine similarity'], reverse=True) 
  all_top_10_lists.append(top_list[1:11])

all_top_10_lists[0]

sentenca 1
sentenca 10
sentenca 100
sentenca 101
sentenca 102
sentenca 103
sentenca 104
sentenca 105
sentenca 106
sentenca 107
sentenca 108
sentenca 109
sentenca 11
sentenca 110
sentenca 111
sentenca 112
sentenca 113
sentenca 114
sentenca 115
sentenca 116
sentenca 117
sentenca 118
sentenca 119
sentenca 12
sentenca 120
sentenca 121
sentenca 122
sentenca 123
sentenca 124
sentenca 125
sentenca 126
sentenca 127
sentenca 128
sentenca 129
sentenca 13
sentenca 130
sentenca 131
sentenca 132
sentenca 133
sentenca 134
sentenca 135
sentenca 136
sentenca 137
sentenca 138
sentenca 139
sentenca 14
sentenca 140
sentenca 141
sentenca 142
sentenca 143
sentenca 144
sentenca 145
sentenca 146
sentenca 147
sentenca 148
sentenca 149
sentenca 15
sentenca 150
sentenca 151
sentenca 152
sentenca 153
sentenca 154
sentenca 155
sentenca 156
sentenca 157
sentenca 158
sentenca 159
sentenca 16
sentenca 160
sentenca 161
sentenca 162
sentenca 163
sentenca 164
sentenca 165
sentenca 166
sentenca 167
sentenca 168
sentenca

KeyError: ignored

In [ ]:
all_top_10 = pd.DataFrame()

for i in range(len(df)):
  new_df = pd.DataFrame.from_dict(all_top_10_lists[i])
  all_top_10 = pd.concat([all_top_10, new_df])

print('Top 10 textos mais semelhantes ao texto \'sentença 1:\'')
all_top_10.head(10)